# Peformance comparison of different algorithms

In this notebook we analyze the performance of various algorithms, including MLPs, dense dynamical systems, and various sorts of sparse dynamical systems.

In [9]:
import torch
import warnings
import time
warnings.filterwarnings("ignore")
# TODO Add these imports and test MaskeLinear and SparseLinear
from iterativennsimple.MaskedLinear import MaskedLinear
from iterativennsimple.SparseLinear import SparseLinear

In [10]:
def generate(size, entries, device="cuda"):
    """create a variety of sparse matrices 

    Args:
        size (int, optional): Size of the square matrix. Defaults to 1000.
        entries (int, optional): Total number of non-zero entries. Note this is an upper bound, but should be close to the actual size. Defaults to 23*1000.
        device (str, optional): "cuda" or "cpu". Defaults to "cuda".

    Returns:
        dict: Dictionary of the sparse matrices
    """ 
    output = {}
    # We first create a COO tensor since that is easier to create
    indices = torch.randint(0, size, (entries,2))
    vals = torch.randn(entries)
    coo = torch.sparse_coo_tensor(indices.t(), vals, (size, size), device=device)
    coo = coo.coalesce()

    # Then we convert it to CSC, CSR and dense
    dense = coo.to_dense()
    csc = coo.to_sparse_csc()
    csr = coo.to_sparse_csr()
    
    # We also create the MaskedLinear and SparseLinear objects
    class moduleWrapper(object):
        def __init__(self, module, device=device):
            self.module = module
            self.device = device
        def __matmul__(self, x):
            return self.module(x.T).T
        
    maskedLinear = moduleWrapper(MaskedLinear.from_coo(coo).to(device), device)
    sparseLinear = moduleWrapper(SparseLinear.from_coo(coo).to(device), device)

    output["dense"] = dense
    output["coo"] = coo
    output["csc"] = csc
    output["csr"] = csr
    output["maskedLinear"] = maskedLinear
    output["sparseLinear"] = sparseLinear
    return output

In [11]:
## check that all of the matrices are the same operator
def matrix_check(size, entries, device):
    # Generate the matrices
    matrices = generate(size, entries, device=device)
    # Size of the RHS
    x_cols = 100
    x = torch.randn(size, x_cols, device=device)

    print('Matrix-matrix multiplication check')
    y_true = None
    base_name = None

    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        y = matrix @ x
        if y_true is None:
            y_true = y
            base_name = matrix_name
        else:
            # print the frobenius norm of the difference
            print(f"||{matrix_name} - {base_name}||_F: {torch.norm(y-y_true)}")
            

In [12]:
matrix_check(1000, 23*1000, "cuda")

Matrix-matrix multiplication check
||coo - dense||_F: 0.00014554106746800244
||csc - dense||_F: 0.0001455377641832456
||csr - dense||_F: 0.00014554393419530243
||maskedLinear - dense||_F: 0.0001326123165199533
||sparseLinear - dense||_F: 0.0001671774371061474


In [15]:
def run_timing(size, entries, device, syncgpu):
    # test if cuda is available
    if device == "cuda":
        if not torch.cuda.is_available():
            print("CUDA is not available, using CPU instead")
            device = "cpu"

    print(f"Running on {device}")
    print(f"Size: {size}")
    print(f"Entries: {entries}")
    print(f"Synchronize GPU: {syncgpu}")
    
    # Generate the matrices
    matrices = generate(size, entries, device=device)

    # Size of the RHS
    x_cols = 100

    # Compute the timings
    print('Matrix-matrix multiplication timings:')
    base_time = None
    base_name = None
    all_times = {}
    for matrix, matrix_name in zip(matrices.values(), matrices.keys()):
        # first, do a few runs to warm up the cache
        for i in range(2):
            x = torch.randn(size, x_cols, device=device)
            y = matrix @ x
        # now do the timings
        matrix_times = []
        for i in range(5):
            x = torch.randn(size, x_cols, device=device)    
            if syncgpu:
                torch.cuda.synchronize()
            start = time.perf_counter()
            y = matrix @ x
            if syncgpu:
                torch.cuda.synchronize()
            matrix_time = time.perf_counter()-start
            matrix_times.append(matrix_time)
        avg_matrix_time = sum(matrix_times)/len(matrix_times)
        min_matrix_time = min(matrix_times)
        max_matrix_time = max(matrix_times)
        if base_time is None:
            base_time = avg_matrix_time
            base_name = matrix_name

        all_times[matrix_name] = avg_matrix_time

        print(f"{matrix_name} avg time:", avg_matrix_time)
        print(f"{matrix_name} min time:", min_matrix_time)
        print(f"{matrix_name} max time:", max_matrix_time)
        print(f"{matrix_name} speedup over {base_name}:", base_time/avg_matrix_time)

    return all_times

In [33]:
all_times = run_timing(30000, 100*30000, "cuda", True)

Running on cuda
Size: 30000
Entries: 3000000
Synchronize GPU: True
Matrix-matrix multiplication timings:
dense avg time: 0.004746346827596426
dense min time: 0.004726625047624111
dense max time: 0.004778423812240362
dense speedup over dense: 1.0
coo avg time: 0.0006691391579806804
coo min time: 0.0006040823645889759
coo max time: 0.0006932811811566353
coo speedup over dense: 7.093213378693739
csc avg time: 0.0022183618508279323
csc min time: 0.002207833807915449
csc max time: 0.0022247261367738247
csc speedup over dense: 2.139572868071548
csr avg time: 0.0003489457070827484
csr min time: 0.0003478880971670151
csr max time: 0.00034957099705934525
csr speedup over dense: 13.60196366155863
maskedLinear avg time: 0.028237955830991267
maskedLinear min time: 0.028230339288711548
maskedLinear max time: 0.028246068861335516
maskedLinear speedup over dense: 0.16808393837018795
sparseLinear avg time: 0.005656864680349827
sparseLinear min time: 0.005647876299917698
sparseLinear max time: 0.005663

In [34]:
import pandas as pd

# Create a dataframe with row and column names for the heatmap
df = pd.DataFrame(columns=all_times.keys(), index=all_times.keys())

# Fill in df with the values we want to display
for name1, time1 in all_times.items():
    for name2, time2 in all_times.items():
        df.loc[name1, name2] = time1/time2
fig = px.imshow(df, text_auto=True, color_continuous_scale='RdBu')
fig.show()